# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews
import numpy as np

# Import API key
from api_keys import geoapify_key

c:\Users\seyhr\anaconda3\envs\dev\lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\seyhr\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\seyhr\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,10.11,93,100,1.82,CL,1734312410
1,1,uturoa,-16.7333,-151.4333,26.49,81,95,4.05,PF,1734312411
2,2,grytviken,-54.2811,-36.5092,5.54,72,100,1.93,GS,1734312412
3,3,adamstown,-25.0660,-130.1015,22.74,67,79,4.21,PN,1734312413
4,4,shahba,32.8551,36.6290,5.65,26,8,2.56,SY,1734312414


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.


In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    geo=True,
    tiles="OSM",
    color="City",
    width = 1000,
    height = 600,
    alpha=0.6,
    hover_cols=["City", "Country", "Humidity"])

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
199,199,corumba,-19.0092,-57.6533,24.37,59,0,0.94,BR,1734312637
268,268,maun,-19.9833,23.4167,25.98,50,0,2.57,BW,1734312715
359,359,salalah,17.0151,54.0924,24.05,35,0,3.60,OM,1734312818
398,398,kara,9.5511,1.1861,21.17,19,0,1.68,TG,1734312861
424,424,leesville,31.1435,-93.2610,21.13,83,0,3.60,US,1734312889
513,513,jacmel,18.2342,-72.5347,24.19,84,0,2.46,HT,1734312995


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
199,corumba,BR,-19.0092,-57.6533,59,
268,maun,BW,-19.9833,23.4167,50,
359,salalah,OM,17.0151,54.0924,35,
398,kara,TG,9.5511,1.1861,19,
424,leesville,US,31.1435,-93.2610,83,
513,jacmel,HT,18.2342,-72.5347,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "limit" : 1,
    "apiKey" : geoapify_key,
    "radius" : radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
corumba - nearest hotel: Hotel El Shadday
maun - nearest hotel: Center Lodge Conference Center
salalah - nearest hotel: Muscat International Hotel
kara - nearest hotel: Hotel Kara
leesville - nearest hotel: Surestay Hotel by Best Western Leesville
jacmel - nearest hotel: Hotel L'Amenitie


,City,Country,Lat,Lng,Humidity,Hotel Name
199,corumba,BR,-19.0092,-57.6533,59,Hotel El Shadday
268,maun,BW,-19.9833,23.4167,50,Center Lodge Conference Center
359,salalah,OM,17.0151,54.0924,35,Muscat International Hotel
398,kara,TG,9.5511,1.1861,19,Hotel Kara
424,leesville,US,31.1435,-93.2610,83,Surestay Hotel by Best Western Leesville
513,jacmel,HT,18.2342,-72.5347,84,Hotel L'Amenitie


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    size="Humidity",
    geo=True,
    tiles="OSM",
    color="City",
    alpha=0.6,
    width = 1000,
    height = 600,
    hover_cols=["City", "Country", "Humidity","Hotel Name"]  # Info to display on hover
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)